# **A Genetic Algorithm for Optimization Of Supply Chain Logistics Network**

In [ ]:
import numpy as np 
import random


In [118]:
class parameters :
  nb_plants = 2 # number of plants
  nb_distrib = 2 # number of distributors
  nb_retailers = 3 # number of retailers
  N = 1000 # population size
  p_c = 0.6 # crossover probability
  p_m = 0.01 # mutation probability
  pr = 1 # unit price


In [121]:
#cost generation :

def generate_capacity(k):
  capacity=[]
  for i in range(k):
    c=int(input("give capacity for plant n°"+str(i)+" :"))
    capacity.append(c)
  return capacity
def generate_demand(i):
  demand=[]
  for k in range(i):
    d=int(input("give demand for retailer n°"+str(k)+" :"))
    demand.append(d)
  return demand
def init_cost(k,j):
  cost=np.empty(shape=(k, j), dtype=float)
  for p in range(k):
    for s in range(j):
      cost[p][s]=int(input("give shipement cost X"+str(p)+" "+str(s)+" : "))
  return cost

capacity=generate_capacity(parameters.nb_plants)

demand=generate_demand(parameters.nb_retailers)

cost_k_j=init_cost(parameters.nb_plants,parameters.nb_distrib)
cost_j_i=init_cost(parameters.nb_distrib,parameters.nb_retailers)




give capacity for plant n°0 :150000
give capacity for plant n°1 :50000
give demand for retailer n°0 :50000
give demand for retailer n°1 :100000
give demand for retailer n°2 :50000
give shipement cost X0 0 : 0
give shipement cost X0 1 : 5
give shipement cost X1 0 : 4
give shipement cost X1 1 : 2
give shipement cost X0 0 : 3
give shipement cost X0 1 : 4
give shipement cost X0 2 : 5
give shipement cost X1 0 : 2
give shipement cost X1 1 : 1
give shipement cost X1 2 : 2


# **Initialization function**

In [122]:
def generate_chromosome():
  plant_dist_weights = [0 for _ in range(parameters.nb_plants*parameters.nb_distrib)] 
  dist_stock = [0 for _ in range(parameters.nb_distrib)]
  for plant in range(parameters.nb_plants):
        plant_capacity=capacity[plant] 
        capacity_part = plant_capacity//(1000*parameters.nb_distrib)
        while(not(plant_capacity==0)):
          for dist in range(parameters.nb_distrib):
            random_nb= np.random.randint(0,100*parameters.nb_distrib)
            plant_dist_weights[plant*parameters.nb_distrib+dist]+= min(capacity_part * random_nb,plant_capacity)
            dist_stock[dist]+=min(capacity_part * random_nb,plant_capacity)
            plant_capacity-=min(capacity_part * random_nb,plant_capacity)
  temp_demand=demand.copy()

  dist_retail_weights = [0 for _ in range(parameters.nb_distrib*parameters.nb_retailers)]

  dist_stock_over=False

  for ret in range(parameters.nb_retailers):
    ret_portion = temp_demand[ret] //(1000*parameters.nb_distrib)
    dist_stock_over=False
    while(not(temp_demand[ret]==0)):
      for dist in range(parameters.nb_distrib):
        if(dist_stock[dist]==0):
          dist_stock_over=True
        random_nb= np.random.randint(0,100*parameters.nb_distrib)
        tmp = min(temp_demand[ret],min(ret_portion*random_nb,dist_stock[dist]))
        dist_retail_weights[dist*parameters.nb_retailers+ret]+=tmp
        temp_demand[ret]-=tmp
        dist_stock[dist]-=tmp
  if(not dist_stock_over):
    for dist in range(parameters.nb_distrib):
      if(dist_stock[dist]==0):
        continue 
      else :
        dist_portion = dist_stock[dist] //(100*parameters.nb_retailers)
        
        while(not(dist_stock[dist]==0)):
          for ret in range(parameters.nb_retailers):
            random_nb= np.random.randint(0,100*parameters.nb_retailers)
            tmp = min(dist_portion*random_nb,dist_stock[dist])
            dist_retail_weights[dist*parameters.nb_retailers+ret]+=tmp
            dist_stock[dist]-=tmp

    
  chromosome = plant_dist_weights+dist_retail_weights 

  return chromosome

In [123]:
def init_population(N):
  population=[generate_chromosome() for _ in range(N)]
  return population


In [124]:
def mutation(chromosome,k,j,i,pm):
    for s in range(k):
      r=np.random.random()
      if r>pm :
        x1=np.random.randint(s*j,s*j + j)
        y1=np.random.randint(s*j,s*j + j)
        l=chromosome[x1]
        chromosome[x1]=chromosome[y1]
        chromosome[y1]=l
      r=np.random.random()
      if r>pm :
        x1=np.random.randint(s*j,s*j + j)
        y1=np.random.randint(s*j,s*j + j)
        if(not(x1==y1)):
          l1=chromosome[x1]
          chromosome[x1]/=2
          chromosome[y1]+=l1/2
    for s in range(j):
      r=np.random.random()
      if r>pm :
        x1=np.random.randint(k*j+s*i,k*j+s*i + i)
        y1=np.random.randint(k*j+s*i,k*j+s*i + i)
        l=chromosome[x1]
        chromosome[x1]=chromosome[y1]
        chromosome[y1]=l
      r=np.random.random()
      if r>pm :
        x1=np.random.randint(k*j+s*i,k*j+s*i + i)
        y1=np.random.randint(k*j+s*i,k*j+s*i + i)
        if(not(x1==y1)):
          l1=chromosome[x1]
          chromosome[x1]/=2
          chromosome[y1]+=l1/2
  
  


In [125]:
#Id matrice ligne contenant les demandes des retailers
#C matrice ligne contenant les capacites des plants
def feasible_mutation(chromosome,k,j,i,pm): 
  mutated_chromosome=chromosome.copy()
  mutation(mutated_chromosome,k,j,i,pm)
  #contraites des plants 
  for p in range(k):
    X=0
    for l in range(j):
      X+=mutated_chromosome[l+p*j]
    if X>capacity[p]: 
      return chromosome 

  #containtes des retailers
  for r in range(i):
    Y=0
    for f in range(j):
      Y+=mutated_chromosome[k*j+f*i+ r]
    if Y<demand[r]:
      return chromosome
  

  #contraites egalites
  for d in range(j):
    X=0
    Y=0 
    for s in range(0,j*k,j):
      X+=mutated_chromosome[s+d]
    for q in range(i):
      Y+=mutated_chromosome[k*j+q+d*i]
    if X!=Y:
      return chromosome
  
  mutated_chromosome_fitness = fitness(mutated_chromosome,k,j,i,parameters.p_m)
  chromosome_fitness = fitness(chromosome,k,j,i,parameters.p_m)
  if (mutated_chromosome_fitness<chromosome_fitness):
    return mutated_chromosome
  else:
    return chromosome
    
  

In [126]:

def fitness(chromosome,k,j,i,pr):
  f=0
  for p in range(k):
    for s in range(j):
      f+=chromosome[p*j+s]*(cost_k_j[p][s]+pr) 
  for p in range(j):
    for s in range(i):
      f+=chromosome[k*j + p*i + s]*cost_j_i[p][s] 
  return f

def get_pr(f,chromosome,k,j,i):
  part=0
  for p in range(k):
    for s in range(j):
      part+=chromosome[p*j+s]*cost_k_j[p][s]
  for p in range(j):
    for s in range(i):
      part+=chromosome[k*j + p*i + s]*cost_j_i[p][s] 
  dot_pr=0
  for p in range(k):
    for s in range(j):
      dot_pr+=chromosome[p*j+s]
  return (f-part)/dot_pr
def tournament_selection(k,j,i,pr,population):
  # list of fitness of each individual
  fit = [fitness(x,k,j,i,pr) for x in population]
  selected_parents=[]
  
  while(not(len(selected_parents)==2)):
    ind_1,ind_2=np.random.randint(0,len(population)-1),np.random.randint(0,len(population)-1)
    if(abs(ind_1-ind_2)>4 and abs(ind_1-ind_2)<0.1*parameters.N):
      subset = population[min(ind_1,ind_2):max(ind_1,ind_2)]
      sub_fit = fit[min(ind_1,ind_2):max(ind_1,ind_2)]
      choice = np.argmin(sub_fit)
      selected_parents.append(subset[choice])

  return selected_parents

def Crossover(parents,K,J,I):

  child1 = parents[0][:K*J] + parents[1][K*J:]
  child2 = parents[1][:K*J] + parents[0][K*J:]
  
  return [child1,child2]

In [127]:
def train_population(population,k,j,i,pc,pr):
  new_population=[]
  child_pop_size = int(pc*len(population))
  child_pop=[]
  cnt_not_found=0
  while(not(len(child_pop)>child_pop_size)):
    parents=tournament_selection(parameters.nb_plants,parameters.nb_distrib,parameters.nb_retailers,parameters.pr,population)
    child1,child2=Crossover(parents,parameters.nb_plants,parameters.nb_distrib,parameters.nb_retailers)
    if(child1==child2):
      child1=feasible_mutation(child1,parameters.nb_plants,parameters.nb_distrib,parameters.nb_retailers,parameters.p_m)
      if(child1 in child_pop):
        cnt_not_found+=1
      else:
        child_pop.append(child1)
    else :
      child1,child2=feasible_mutation(child1,parameters.nb_plants,parameters.nb_distrib,parameters.nb_retailers,parameters.p_m),feasible_mutation(child2,parameters.nb_plants,parameters.nb_distrib,parameters.nb_retailers,parameters.p_m)
      if(child1 in child_pop and child2 in child_pop):
        cnt_not_found+=1
      if(child1 not in child_pop):
        child_pop.append(child1)
      if(child2 not in child_pop):
        child_pop.append(child2)
      
    if(cnt_not_found==50):
      break
    
  
  new_population+=population
  new_population+=child_pop
  new_pop_fitness =[fitness(x,k,j,i,pr) for x in new_population]

  combined_list = list(zip(new_pop_fitness, new_population))
  sorted_list = sorted(combined_list, reverse=False)
  sorted_fitness, sorted_population = zip(*sorted_list)
  return sorted_population[:parameters.N]

In [132]:
def generate_best_solution(nb_gens):
  population = init_population(parameters.N)
  init_fit=[fitness(x,parameters.nb_plants,parameters.nb_distrib,parameters.nb_retailers,parameters.pr) for x in population]
  print("initial minimal fitness",min(init_fit))
  best_pop=[]
  best_fit=100000000
  for i in range(nb_gens):
    print("Training generation n°: "+str(i))
    population=train_population(population,parameters.nb_plants,parameters.nb_distrib,parameters.nb_retailers,parameters.p_c,parameters.pr)
    fit = [fitness(x,parameters.nb_plants,parameters.nb_distrib,parameters.nb_retailers,parameters.pr) for x in population]
    cur_best = np.argmin(fit)
    if(fit[cur_best]<best_fit):
      best_fit=fit[cur_best]
      best_pop=population[cur_best]
      print("current best sol: "+str(best_fit))
    #display(population)
  
  return best_pop,best_fit

In [ ]:
generate_best_solution(1000)

initial minimal fitness 1169275.0
Training generation n°: 0
current best sol: 1084625.0
Training generation n°: 1
current best sol: 1044050.0
Training generation n°: 2
Training generation n°: 3
Training generation n°: 4
Training generation n°: 5
Training generation n°: 6
Training generation n°: 7
Training generation n°: 8
Training generation n°: 9
Training generation n°: 10
Training generation n°: 11
Training generation n°: 12
Training generation n°: 13
Training generation n°: 14
Training generation n°: 15
Training generation n°: 16
Training generation n°: 17
Training generation n°: 18
Training generation n°: 19
Training generation n°: 20
Training generation n°: 21
Training generation n°: 22
Training generation n°: 23
Training generation n°: 24
Training generation n°: 25
Training generation n°: 26
Training generation n°: 27
Training generation n°: 28
Training generation n°: 29
Training generation n°: 30
Training generation n°: 31
Training generation n°: 32
Training generation n°: 33
Tr